In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras 
from tensorflow.keras.utils import to_categorical
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import ParameterGrid
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
label_count = np.unique(y_train)
print("Total number of labels:", label_count.size)

Total number of labels: 10


In [4]:
x_train.shape

(60000, 28, 28)

In [5]:
#Transform the shape of the data from (60000, 28, 28) to (60000, 784)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])


In [6]:
#After converting the data to from 3d to 2d
print("Number of training samples:", x_train.shape[0], "and shape of each sample:", x_train.shape[1])
print("Number of testing samples:", x_test.shape[0], "and shape of each sample:", x_test.shape[1])

Number of training samples: 60000 and shape of each sample: 784
Number of testing samples: 10000 and shape of each sample: 784


In [7]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

Normalize the data


In [8]:
#Normalize the data. Max value is 255 and min value is 0.
x_train = x_train/255
x_test = x_test/255


One hot encoding


In [9]:
#Inititially before one hot encoding our classes look like this
print(y_train[0])

#After one hot encoding our classes look like this
y_train = to_categorical(y_train, label_count.size)
y_test = to_categorical(y_test, label_count.size)

print(y_train[0])


5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [18]:
#Split the data into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

Building the MLP model

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout


In [11]:
output_dim = label_count.size
input_dim = x_train.shape[1]

batch_size = 128
nb_epoch = 10


In [12]:
model = Sequential()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-04-10 15:36:57.488430: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-10 15:36:57.488866: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax'))

In [14]:
# Before training a model, you need to configure the learning process, which is done via the compile method

# It receives three arguments:
# An optimizer. This could be the string identifier of an existing optimizer , https://keras.io/optimizers/
# A loss function. This is the objective that the model will try to minimize., https://keras.io/losses/
# A list of metrics. For any classification problem you will want to set this to metrics=['accuracy'].  https://keras.io/metrics/

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# fit() function Trains the model for a fixed number of epochs (iterations on a dataset).
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_val, y_val))

Epoch 1/10
 30/375 [=>............................] - ETA: 1s - loss: 0.3802 - accuracy: 0.9029

2022-04-10 15:57:06.102805: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 3s 7ms/step - loss: 0.3980 - accuracy: 0.8924 - val_loss: 0.3880 - val_accuracy: 0.8951
Epoch 2/10
375/375 [==============================] - 3s 7ms/step - loss: 0.3921 - accuracy: 0.8932 - val_loss: 0.3824 - val_accuracy: 0.8957
Epoch 3/10
375/375 [==============================] - 3s 7ms/step - loss: 0.3867 - accuracy: 0.8949 - val_loss: 0.3775 - val_accuracy: 0.8961
Epoch 4/10
375/375 [==============================] - 3s 7ms/step - loss: 0.3819 - accuracy: 0.8958 - val_loss: 0.3733 - val_accuracy: 0.8973
Epoch 5/10
375/375 [==============================] - 3s 7ms/step - loss: 0.3775 - accuracy: 0.8969 - val_loss: 0.3694 - val_accuracy: 0.8985
Epoch 6/10
375/375 [==============================] - 2s 7ms/step - loss: 0.3735 - accuracy: 0.8978 - val_loss: 0.3659 - val_accuracy: 0.8991
Epoch 7/10
375/375 [==============================] - 2s 7ms/step - loss: 0.3698 - accuracy: 0.8987 - val_loss: 0.3625 - val_accuracy: 0.9003
Epoch 8/10
375/37

MLP + Relu + ADAM

In [20]:
model_relu = Sequential()
model_relu.add(Dense(512, input_dim=input_dim, activation='relu'))
model_relu.add(Dense(output_dim, activation='relu'))
model_relu.add(Dense(output_dim, activation='softmax'))

model_relu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model_relu.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_val, y_val))

Epoch 1/10
 16/375 [>.............................] - ETA: 2s - loss: 1.9852 - accuracy: 0.2666

2022-04-10 15:57:31.541235: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


371/375 [============================>.] - ETA: 0s - loss: 0.6309 - accuracy: 0.7958

2022-04-10 15:57:34.099848: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 3s 8ms/step - loss: 0.6268 - accuracy: 0.7973 - val_loss: 0.2221 - val_accuracy: 0.9392
Epoch 2/10
375/375 [==============================] - 3s 8ms/step - loss: 0.1677 - accuracy: 0.9540 - val_loss: 0.1376 - val_accuracy: 0.9602
Epoch 3/10
375/375 [==============================] - 3s 7ms/step - loss: 0.1070 - accuracy: 0.9697 - val_loss: 0.1112 - val_accuracy: 0.9667
Epoch 4/10
375/375 [==============================] - 3s 8ms/step - loss: 0.0763 - accuracy: 0.9787 - val_loss: 0.0958 - val_accuracy: 0.9711
Epoch 5/10
375/375 [==============================] - 3s 8ms/step - loss: 0.0580 - accuracy: 0.9833 - val_loss: 0.0877 - val_accuracy: 0.9751
Epoch 6/10
375/375 [==============================] - 3s 7ms/step - loss: 0.0452 - accuracy: 0.9873 - val_loss: 0.0826 - val_accuracy: 0.9761
Epoch 7/10
375/375 [==============================] - 3s 8ms/step - loss: 0.0344 - accuracy: 0.9900 - val_loss: 0.0867 - val_accuracy: 0.9744
Epoch 8/10
375/37

In [21]:
#Check accuracy of the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.3404335677623749
Test accuracy: 0.9080000519752502


MLP + Sigmoid + ADAM

In [22]:
model_relu = Sequential()
model_relu.add(Dense(512, input_dim=input_dim, activation='sigmoid'))
model_relu.add(Dense(output_dim, activation='sigmoid'))
model_relu.add(Dense(output_dim, activation='softmax'))

model_relu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model_relu.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_val, y_val))

Epoch 1/10
 17/375 [>.............................] - ETA: 2s - loss: 2.2134 - accuracy: 0.2321

2022-04-10 16:01:27.060571: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


372/375 [============================>.] - ETA: 0s - loss: 1.4217 - accuracy: 0.7149

2022-04-10 16:01:29.587789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 3s 8ms/step - loss: 1.4183 - accuracy: 0.7160 - val_loss: 0.9794 - val_accuracy: 0.8593
Epoch 2/10
375/375 [==============================] - 3s 8ms/step - loss: 0.7257 - accuracy: 0.8897 - val_loss: 0.5319 - val_accuracy: 0.9169
Epoch 3/10
375/375 [==============================] - 3s 8ms/step - loss: 0.4257 - accuracy: 0.9280 - val_loss: 0.3433 - val_accuracy: 0.9369
Epoch 4/10
375/375 [==============================] - 3s 8ms/step - loss: 0.2917 - accuracy: 0.9442 - val_loss: 0.2584 - val_accuracy: 0.9466
Epoch 5/10
375/375 [==============================] - 3s 8ms/step - loss: 0.2179 - accuracy: 0.9559 - val_loss: 0.2019 - val_accuracy: 0.9557
Epoch 6/10
375/375 [==============================] - 3s 8ms/step - loss: 0.1705 - accuracy: 0.9642 - val_loss: 0.1719 - val_accuracy: 0.9607
Epoch 7/10
375/375 [==============================] - 3s 8ms/step - loss: 0.1365 - accuracy: 0.9711 - val_loss: 0.1475 - val_accuracy: 0.9646
Epoch 8/10
375/37

In [23]:
#Check accuracy of the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.3404335677623749
Test accuracy: 0.9080000519752502
